In [ ]:
# !pip install opencv-python
# !pip install pillow
# !pip install scipy

In [12]:
import numpy as np
from imageio import imread, imsave
from PIL import Image

In [13]:

image1 = imread('i1.png', as_gray=True)
image2 = imread('i2.png', as_gray=True)

In [14]:
def get_diff(image1, image2):
  size = np.asarray(image1.shape) / 5
  size = size.astype(int) * 5
  image1 = np.resize(np.asarray(image1), (size)).astype(np.int16)
  image2 = np.resize(np.asarray(image2), (size)).astype(np.int16)
  diff = abs(image1 - image2)
  return diff

In [15]:
def find_vector_set(diff, size):

    i, j = 0, 0
    vector_set = np.zeros((int(size[0] * size[1] / 25), 25))
    while i < vector_set.shape[0]:
        while j < size[0]:
            k = 0
            while k < size[1]:
                vector_set[i, :] = diff[j:j+5, k:k+5].ravel()
                k += 5
            j += 5
        i += 1

    # Normalizing
    mean_v = np.mean(vector_set, axis=0)
    vector_set = vector_set - mean_v

    return vector_set, mean_v


In [16]:
diff = get_diff(image1, image2)
VS, MV = find_vector_set(diff, size)

In [17]:
VS

array([[ 4.99951942e+00,  1.09989427e+01,  1.49985582e+01, ...,
         1.69983660e+01,  3.99961553e+01,  4.69954825e+01],
       [-4.80584391e-04, -1.05728566e-03, -1.44175317e-03, ...,
        -1.63398693e-03, -3.84467512e-03, -4.51749327e-03],
       [-4.80584391e-04, -1.05728566e-03, -1.44175317e-03, ...,
        -1.63398693e-03, -3.84467512e-03, -4.51749327e-03],
       ...,
       [-4.80584391e-04, -1.05728566e-03, -1.44175317e-03, ...,
        -1.63398693e-03, -3.84467512e-03, -4.51749327e-03],
       [-4.80584391e-04, -1.05728566e-03, -1.44175317e-03, ...,
        -1.63398693e-03, -3.84467512e-03, -4.51749327e-03],
       [-4.80584391e-04, -1.05728566e-03, -1.44175317e-03, ...,
        -1.63398693e-03, -3.84467512e-03, -4.51749327e-03]])

In [6]:
from sklearn.decomposition import PCA

def fit_pca(VS):
  pca = PCA()
  pca.fit(VS)
  eigenvalues = pca.components_

In [7]:
eigenvalues[1]

array([-0.00438653, -0.08035527,  0.00909561, -0.11042497,  0.00165256,
        0.00909561,  0.08087836,  0.00925426,  0.16175672,  0.29947243,
        0.05288183,  0.12143185, -0.10552912,  0.05288183, -0.22084995,
        0.00330511,  0.29947243, -0.52373075,  0.20738429, -0.52373075,
        0.        , -0.11042497,  0.20142829,  0.03701705,  0.20738429])

In [8]:
def find_FinalVS(eigenvalues, diff, MV, size):

    i = 2
    feature_vector_set = []

    while i < size[0] - 2:
        j = 2
        while j < size[1] - 2:
            block = diff[i-2:i+3, j-2:j+3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j+1
        i = i+1

    FinalVS = np.dot(feature_vector_set, eigenvalues)
    FinalVS = FinalVS - MV
    return FinalVS

FinalVS = find_FinalVS(eigenvalues, diff, MV, size)

In [9]:
FinalVS

array([[-25.08341385, -23.35176326,  14.60490459, ...,   9.38574602,
         33.46482257,   0.33173248],
       [-19.75046484, -25.37914343,  22.08353984, ...,   8.51097444,
         19.09111262,  -5.69180443],
       [-18.89648551, -25.87670823,  14.42308733, ...,  12.535537  ,
         18.7426567 ,  -5.06893626],
       ...,
       [-13.98530718, -13.40248389,  -1.15819876, ...,  13.74878649,
         11.36494485,  16.77857368],
       [-24.23360521, -21.95808774,   3.55062493, ...,  17.9617827 ,
         23.83671082,  22.27658602],
       [-32.17674043, -25.26250031,  18.51122745, ...,  18.98627771,
         27.85910178,  20.26184503]])

In [10]:
from sklearn.cluster import KMeans
from collections import Counter


def clustering(FVS, components, new):

    kmeans = KMeans(components, verbose=0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count = Counter(output)

    least_index = min(count, key=count.get)
    change_map = np.reshape(output, (new[0] - 4, new[1] - 4))
    
    return least_index, change_map


components = 3
least_index, change_map = clustering(FinalVS, components, size)
imsave("changemap.jpg", change_map)

In [11]:
import cv2

change_map[change_map == least_index] = 255
change_map[change_map != 255] = 0

change_map = change_map.astype(np.uint8)
kernel = np.asarray(((0, 0, 1, 0, 0),
                     (0, 1, 1, 1, 0),
                     (1, 1, 1, 1, 1),
                     (0, 1, 1, 1, 0),
                     (0, 0, 1, 0, 0)), dtype=np.uint8)
cleanChangeMap = cv2.erode(change_map, kernel)

imsave("changemap.jpg", change_map)
imsave("cleanchangemap.jpg", cleanChangeMap)